- 前処理
    - 線形モデル用

# Library

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
import datetime

import os

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# model
import lightgbm as lgb

# shap
import shap

# my_tool
import sys
sys.path.append('../../')
import importlib
from utils import my_ds_tools as mdt
importlib.reload(mdt)

<module 'utils.my_ds_tools' from 'D:\\workspace\\kaggle_practice\\house_prices_advanced_regression_techniques\\src_analysis\\../..\\utils\\my_ds_tools.py'>

# Load data

In [2]:
base_df = pd.read_csv("../input/train.csv")
test_df = pd.read_csv("../input/test.csv")
base_df.head(3)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500


In [3]:
base_df.shape

(1460, 81)

# Config

In [4]:
VERSION = 'v1'
USAGE = 'linear'
OUT_DIR = '../intermediate/'
Y_COL = 'SalePrice'
DROP_COLS = ['Id']

# Preprocess

### Categorical
- Street
  - 関係なさそう？
- LotConfig
  - 関係なさそう？
- RoofMatl
  - 変更なし
  - ひょっとしたらWdShnglか否かが効きそう
- ExterQual
  - 変更なし
  - is_Exが効きそう
- BsmtQual
  - 変更なし
  - is_Exが効きそう
- KichenQual
  - is_Exが効きそう

### Numerical
- MSSubClass, OverallQual, OverallCond 
  - カテゴリ変数に変換
- LotArea, LotFrontage
  - 対数化
  - LotFrontageの欠損値：median埋め
- YearRemodAdd
  - is_remodという変数を追加
    - if YearRemodAdd != YearBuilt
  - PastYearsという変数を追加
    - this year - max(YearBuilt, YearRemodAdd)
- MasVnrArea
  - 対数化
  - 欠損値：mode埋め
- GarageYrBlt
  - 欠損値：平均値埋め
- BsmtFinSf2
  - 使わない
- HalfBath
  - 使わない
- KitchenAbvGr
  - 使わない
- YrSold
  - 使わない

### others
- index=1298は外れ値のため、使わない
  - BsmtSF1>5000だったり、TotalBsmtSF>6000だったりと外れ値。


In [5]:
base_df2 = base_df.drop(Y_COL, axis=1)
base_df2['flg'] = 'train'

print(base_df2.shape)
base_df2 = base_df2.query('TotalBsmtSF < 6000')
print(base_df2.shape)

test_df2 = test_df.copy()
test_df2['flg'] = 'test'

df = pd.concat([base_df2, test_df2])

(1460, 81)
(1459, 81)


In [6]:
df.shape

(2918, 81)

## Numerical

In [13]:
numerical = ['MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'FixedMSSubClass', 'FixedOverallQual', 'FixedOverallCond', 'LogLotArea', 'LogLotFrontage', 'IsRemod', 'PastYears']

In [14]:
def is_remodeled(x):
    return x['YearRemodAdd'] != x['YearBuilt']

def past_years(x):
    this_year = datetime.date.today().year#年
    return this_year - max(x['YearBuilt'], x['YearRemodAdd'])

In [15]:
df['IsRemod'] = df.apply(lambda x: is_remodeled(x), axis=1)
df['PastYears'] = df.apply(lambda x: past_years(x), axis=1)

In [16]:
# numerical

In [17]:
num2cat_list = ['MSSubClass', 'OverallQual', 'OverallCond']
for c in num2cat_list:
    df['Fixed' + c] = df[c].astype(str)
    
log_list = ['LotArea', 'LotFrontage']
for c in log_list:
    df['Log' + c] = df[c].map(lambda x: np.log(x) if x > 0 else 0)

In [18]:
train_tmp = df.query('flg == "train"')

med_cols = ['LotFrontage']
mode_cols = ['MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath', 'GarageCars', 'GarageArea']
mean_cols = ['GarageYrBlt']
impute_dict = {}
for c in med_cols:
    impute_dict[c] = train_tmp[c].median()
for c in mode_cols:
    impute_dict[c] = train_tmp[c].mode()[0]
for c in mean_cols:
    impute_dict[c] = train_tmp[c].mean()

df = df.fillna(impute_dict)
df.loc[:, numerical].isnull().sum()

MasVnrArea          0
BsmtFinSF1          0
BsmtFinSF2          0
BsmtUnfSF           0
TotalBsmtSF         0
2ndFlrSF            0
LowQualFinSF        0
GrLivArea           0
BsmtFullBath        0
BsmtHalfBath        0
FullBath            0
HalfBath            0
BedroomAbvGr        0
KitchenAbvGr        0
Fireplaces          0
GarageYrBlt         0
GarageCars          0
GarageArea          0
WoodDeckSF          0
OpenPorchSF         0
EnclosedPorch       0
3SsnPorch           0
ScreenPorch         0
PoolArea            0
MiscVal             0
FixedMSSubClass     0
FixedOverallQual    0
FixedOverallCond    0
LogLotArea          0
LogLotFrontage      0
IsRemod             0
PastYears           0
dtype: int64

In [19]:
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,flg,IsRemod,PastYears,FixedMSSubClass,FixedOverallQual,FixedOverallCond,LogLotArea,LogLotFrontage
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706.0,Unf,0.0,150.0,856.0,GasA,Ex,Y,SBrkr,856,854,0,1710,1.0,0.0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2.0,548.0,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,train,False,19,60,7,5,9.041922,4.174387
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978.0,Unf,0.0,284.0,1262.0,GasA,Ex,Y,SBrkr,1262,0,0,1262,0.0,1.0,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2.0,460.0,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,train,False,46,20,6,8,9.169518,4.382027
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486.0,Unf,0.0,434.0,920.0,GasA,Ex,Y,SBrkr,920,866,0,1786,1.0,0.0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2.0,608.0,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,train,True,20,60,7,5,9.328123,4.219508
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216.0,Unf,0.0,540.0,756.0,GasA,Gd,Y,SBrkr,961,756,0,1717,1.0,0.0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3.0,642.0,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,train,True,52,70,7,5,9.164296,4.094345
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655.0,Unf,0.0,490.0,1145.0,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1.0,0.0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3.0,836.0,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,train,False,22,60,8,5,9.565214,4.430817


## Categorical

In [20]:
flg_drop_cols = DROP_COLS.copy()
flg_drop_cols.append('flg')
x = df.drop(flg_drop_cols, axis=1)
y = df.loc[:, 'flg']
prepro_tool = mdt.DS_PREPROCESS(x, y)
desc_features = prepro_tool.get_desc_features()

In [21]:
for c in desc_features['cat']:
    print(c)

MSZoning
Street
Alley
LotShape
LandContour
Utilities
LotConfig
LandSlope
Neighborhood
Condition1
Condition2
BldgType
HouseStyle
RoofStyle
RoofMatl
Exterior1st
Exterior2nd
MasVnrType
ExterQual
ExterCond
Foundation
BsmtQual
BsmtCond
BsmtExposure
BsmtFinType1
BsmtFinType2
Heating
HeatingQC
CentralAir
Electrical
KitchenQual
Functional
FireplaceQu
GarageType
GarageFinish
GarageQual
GarageCond
PavedDrive
PoolQC
Fence
MiscFeature
SaleType
SaleCondition
FixedMSSubClass
FixedOverallQual
FixedOverallCond


In [22]:
df = pd.get_dummies(df, columns=desc_features['cat'])
df.head()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,flg,IsRemod,PastYears,LogLotArea,LogLotFrontage,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Grvl,Street_Pave,Alley_Grvl,...,SaleType_COD,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,FixedMSSubClass_120,FixedMSSubClass_150,FixedMSSubClass_160,FixedMSSubClass_180,FixedMSSubClass_190,FixedMSSubClass_20,FixedMSSubClass_30,FixedMSSubClass_40,FixedMSSubClass_45,FixedMSSubClass_50,FixedMSSubClass_60,FixedMSSubClass_70,FixedMSSubClass_75,FixedMSSubClass_80,FixedMSSubClass_85,FixedMSSubClass_90,FixedOverallQual_1,FixedOverallQual_10,FixedOverallQual_2,FixedOverallQual_3,FixedOverallQual_4,FixedOverallQual_5,FixedOverallQual_6,FixedOverallQual_7,FixedOverallQual_8,FixedOverallQual_9,FixedOverallCond_1,FixedOverallCond_2,FixedOverallCond_3,FixedOverallCond_4,FixedOverallCond_5,FixedOverallCond_6,FixedOverallCond_7,FixedOverallCond_8,FixedOverallCond_9
0,1,60,65.0,8450,7,5,2003,2003,196.0,706.0,0.0,150.0,856.0,856,854,0,1710,1.0,0.0,2,1,3,1,8,0,2003.0,2.0,548.0,0,61,0,0,0,0,0,2,2008,train,False,19,9.041922,4.174387,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0
1,2,20,80.0,9600,6,8,1976,1976,0.0,978.0,0.0,284.0,1262.0,1262,0,0,1262,0.0,1.0,2,0,3,1,6,1,1976.0,2.0,460.0,298,0,0,0,0,0,0,5,2007,train,False,46,9.169518,4.382027,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0
2,3,60,68.0,11250,7,5,2001,2002,162.0,486.0,0.0,434.0,920.0,920,866,0,1786,1.0,0.0,2,1,3,1,6,1,2001.0,2.0,608.0,0,42,0,0,0,0,0,9,2008,train,True,20,9.328123,4.219508,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0
3,4,70,60.0,9550,7,5,1915,1970,0.0,216.0,0.0,540.0,756.0,961,756,0,1717,1.0,0.0,1,0,3,1,7,1,1998.0,3.0,642.0,0,35,272,0,0,0,0,2,2006,train,True,52,9.164296,4.094345,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0
4,5,60,84.0,14260,8,5,2000,2000,350.0,655.0,0.0,490.0,1145.0,1145,1053,0,2198,1.0,0.0,2,1,4,1,9,1,2000.0,3.0,836.0,192,84,0,0,0,0,0,12,2008,train,False,22,9.565214,4.430817,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0


In [23]:
df.isnull().sum()

Id                    0
MSSubClass            0
LotFrontage           0
LotArea               0
OverallQual           0
                     ..
FixedOverallCond_5    0
FixedOverallCond_6    0
FixedOverallCond_7    0
FixedOverallCond_8    0
FixedOverallCond_9    0
Length: 328, dtype: int64

# Output

In [24]:
train_df = df.query('flg == "train"').drop('flg', axis=1).merge(base_df[['Id', Y_COL]], on='Id', how='left')
train_df.head(2)

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,IsRemod,PastYears,LogLotArea,LogLotFrontage,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Grvl,Street_Pave,Alley_Grvl,Alley_Pave,...,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,FixedMSSubClass_120,FixedMSSubClass_150,FixedMSSubClass_160,FixedMSSubClass_180,FixedMSSubClass_190,FixedMSSubClass_20,FixedMSSubClass_30,FixedMSSubClass_40,FixedMSSubClass_45,FixedMSSubClass_50,FixedMSSubClass_60,FixedMSSubClass_70,FixedMSSubClass_75,FixedMSSubClass_80,FixedMSSubClass_85,FixedMSSubClass_90,FixedOverallQual_1,FixedOverallQual_10,FixedOverallQual_2,FixedOverallQual_3,FixedOverallQual_4,FixedOverallQual_5,FixedOverallQual_6,FixedOverallQual_7,FixedOverallQual_8,FixedOverallQual_9,FixedOverallCond_1,FixedOverallCond_2,FixedOverallCond_3,FixedOverallCond_4,FixedOverallCond_5,FixedOverallCond_6,FixedOverallCond_7,FixedOverallCond_8,FixedOverallCond_9,SalePrice
0,1,60,65.0,8450,7,5,2003,2003,196.0,706.0,0.0,150.0,856.0,856,854,0,1710,1.0,0.0,2,1,3,1,8,0,2003.0,2.0,548.0,0,61,0,0,0,0,0,2,2008,False,19,9.041922,4.174387,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,208500
1,2,20,80.0,9600,6,8,1976,1976,0.0,978.0,0.0,284.0,1262.0,1262,0,0,1262,0.0,1.0,2,0,3,1,6,1,1976.0,2.0,460.0,298,0,0,0,0,0,0,5,2007,False,46,9.169518,4.382027,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,181500


In [25]:
test_df = df.query('flg == "train"').drop('flg', axis=1)

In [26]:
train_df.to_csv(OUT_DIR + 'train_for_' + USAGE + '_' + VERSION + '.csv')
test_df.to_csv(OUT_DIR + 'test_for_' + USAGE + '_' + VERSION + '.csv')